In [ ]:
pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171048 sha256=f3833514f1e536f3c3de9c6dcc89a53db6ccbdbb48f33849cbcc4f46937fa1cf
  Stored in directory: /root/.cache/pip/wheels/fa/7a/e9/22dd0515e1bad255e51663ee513a2fa839c95934c5fc301090
Successfully built emoji


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
from cleantext import clean

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#df = pd.read_csv('mbti.csv')
data = pd.read_csv('gdrive/My Drive/Colab Notebooks/NLP_personalityTrait/Twitter.csv')

In [ ]:
#data['Tweet']=clean(data['Tweet'],no_emoji=True)
count=0
for i in data['Tweet']:
  data.loc[count,'Tweet']=clean(i, no_emoji=True)
  count+=1

In [ ]:
data.head()

,Unnamed: 0,User Name,Date,Like Count,Tweet
0,0,talktodhee,2021-12-10 19:32:22+00:00,96,excited to release the song ezhundhu va as par...
1,1,talktodhee,2021-08-20 15:58:05+00:00,2260,this is so special. extraaa special to be in t...
2,2,talktodhee,2021-06-27 07:50:41+00:00,4751,thank you for taking the time to wish me happy...
3,3,talktodhee,2021-06-26 13:05:04+00:00,723,thank you uday anna!!
4,4,talktodhee,2021-06-26 13:02:28+00:00,8850,


In [ ]:
# Creating a dataframe wherein each person and his personality type is a row and all the posts by him are the columns
l2 = list(set(data['User Name']))
d = {}
count = 0
for ptype in l2:
    d[ptype] = ''
for pertype in data['User Name']:
    d[pertype] = d[pertype]+"|||"+ data.iloc[count,4]
    count+=1


In [ ]:
df = pd.DataFrame()
for i in d.keys():
    df.loc[i,'posts']= d[i]
df

,posts
talktodhee,|||excited to release the song ezhundhu va as ...
mari_selvaraj,"|||@arunmatheswaran @iamvasanthravi """" . . htt..."
PasupathyMasi,||||||@beemji|||congratulations @arya_offl @sa...


In [ ]:
# converting all to lower case
df['posts'] = [i.lower() for i in df['posts']]

In [ ]:
df['posts'].head()

talktodhee       |||excited to release the song ezhundhu va as ...
mari_selvaraj    |||@arunmatheswaran @iamvasanthravi "" . . htt...
PasupathyMasi    ||||||@beemji|||congratulations @arya_offl @sa...
Name: posts, dtype: object

In [ ]:
df['Distinct Posts'] = [i.split('|||') for i in df['posts']]
#df['Distinct Posts']
df.iat[0,1]

['',
 'excited to release the song ezhundhu va as part of 16 days of activism against domestic violence. lyrics based on poems by survivors of domestic violence. composed by revaa.\ndomestic violence national 24x7 helpline - dhwani @pcvc2000: 1800 102 7282\nhttps://t.co/exjksxdxyw',
 'this is so special. extraaa special to be in this with @shanvdp\nthank you @rollingstonein & @anuragtagat\nall my love and gratitude to you guys x https://t.co/91fcqb1eiu',
 'thank you for taking the time to wish me happy birthday yesterday!! it means so much. they absolutely made my day. all my love',
 'thank you uday anna!!',
 '',
 'thank you vivek anna!!',
 'thank you sooo much!!',
 '@kaviara45785693 aama',
 '',
 'thank you so much @djsnake for this!! this feels surreal. all my love and gratitude to @therukuralarivu @music_santhosh @joinmaajja @spotifyindia and to you all for making enjoy enjaami what it is\nlisten now: https://t.co/akpu5dx9vi',
 '@spotifyindia @djsnake @spotify thanniyil odum paambu k

In [ ]:
df['Number of posts'] = df['Distinct Posts'].apply(len)
df.drop('Distinct Posts',inplace=True,axis=1)
no_posts = df.groupby('Number of posts').count()
no_posts

,posts
Number of posts,
23,1
37,1
182,1


In [ ]:
#removing urls
df['Posts']= df['posts'].apply(lambda x: re.sub(r'https?:[?:A-Za-z0-9//_?.=/-]+', '', x.replace('|','')))
#keeping the end of strings
#removing punctuation marks
df['Posts']=df['Posts'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
#removing numbers and alphanumeric characters
df['Posts']=df['Posts'].apply(lambda x: re.sub(r'\w*\d\w*', '', x))
# removing special symbols
df['Posts']=df['Posts'].apply(lambda x: re.sub(r'[0-9,."\'*.?/\()@#!$%&^+]', '', x))
# removing repetitive multiple letter words and too long or too short words
df['Posts'] = df['Posts'].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x))
df['Posts'] = df['Posts'].apply(lambda x: re.sub(r'(\b\w{0,3})?\b','',x))
df['Posts'] = df['Posts'].apply(lambda x: re.sub(r'(\b\w{30,1000})?\b','',x))
#df['Posts'] = df['Posts'].apply(lambda x: re.sub(r'\s[\s+]','',x))
df.drop('posts',inplace=True,axis=1)
df.to_csv('Twitter_cleaned.csv')

In [ ]:
df

,Number of posts,Posts
talktodhee,37,excited release song ezhundhu part days ...
mari_selvaraj,182,arunmatheswaran iamvasanthravi beemji\ntho...
PasupathyMasi,23,beemjicongratulations aryaoffl saktisounder ik...
